# Element Parameter Detection

## Setup

In [ ]:
%load_ext autoreload

import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch
from numpy.random import default_rng
from tqdm.notebook import tqdm

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path

import ifcopenshell
import open3d as o3d

from src.elements import *
from src.ifc import *
from src.preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance, get_oriented_bbox_from_points
from src.icp import icp_finetuning
from src.chamfer import *
from src.utils import *
from src.plots import plot_error_graph, plot_parameter_errors
from src.pca import testset_PCA
from src.finetune import chamfer_fine_tune, mahalanobis_fine_tune
from src.cloud import add_noise

In [ ]:
random.seed = 42
rng = default_rng()

In [ ]:
# path = Path("ModelNet10")
# path = Path('/content/drive/MyDrive/ElementNet/')
path = Path("output/")
# savepath = '/content/drive/MyDrive/ElementNet/'
savepath = "models/"
cuda = torch.device("cuda")

noise = False

## Model

## Test

Analyze results statistically

POINTNET++

In [ ]:
train_transforms = transforms.Compose(
    [
        Normalize(),
        #                    RandomNoise(),
        ToTensor(),
    ]
)

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath("industrial-facility-relationships/"))
BASE_DIR = os.path.join(BASE_DIR, "pointnet2")
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, "models"))

inference = False
cloi = False

if inference:
    if cloi:
        path = Path("cloi/")
        ext = ".pcd"
    else:
        #     path = Path('output/bp_data/')
        # path = Path('output/east_ref/')
        path = Path("occluded/west/")
        # path = Path('output/')

        # path = Path('/mnt/c/data/3D_CAD/east_clouds/')
        ext = ".pcd"

else:
    if not noise:
        # path = Path("occluded/")
        path = Path("output/")
    else:
        path = Path("output/noisy/")
    ext = ".pcd"

cat = "elbow"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False
cat_targets = {"elbow": 14, "bend": 14, "tee": 19, "pipe": 11, "flange": 13}

if inference:
    test_ds = PointCloudData(
        path,
        valid=True,
        folder="test",
        category=cat,
        transform=train_transforms,
        inference=True,
    )
    targets = cat_targets[cat]
else:
    test_ds = PointCloudData(
        path, valid=True, folder="test", category=cat, transform=train_transforms
    )
    targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = Path("pointnet2/log/classification/pointnet2_cls_ssg/")
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

# checkpoint = torch.load(model_path/'checkpoints/best_model.pth')
checkpoint = torch.load(model_path / "checkpoints/models/best_model_e_chamfer_0005.pth")
# checkpoint = torch.load(model_path/'checkpoints/models/best_model_t_chamfer_00005_bp.pth')
predictor.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
blueprint = "data/sample.ifc"
temp_dir = "output/temp/"
target_dir = "output/tee/test/"

ifcConvert_executable = "scripts/./IfcConvert"
cloudCompare_executable = "cloudcompare.CloudCompare"
sample_size = 2048
threshold = 2

In [ ]:
def model_inference(model, loader, device, calculate_score=False):
    predictor = model.eval()
    predictions_list, pcd_list, id_list = [], [], []
    with torch.no_grad():
        for j, data in tqdm(enumerate(loader), total=len(loader)):
            points, ids = data["pointcloud"].to(device).float(), data["id"].to(device)
            points = points.transpose(2, 1)
            preds, _ = predictor(points)
            preds, points, ids = (
                preds.to(torch.device("cpu")),
                points.to(torch.device("cpu")),
                data["id"].to(torch.device("cpu")),
            )
            for i, pr in enumerate(preds):
                predictions_list.append(pr.numpy())
                pcd_list.append(points[i].numpy())
                id_list.append(ids[i].numpy())

        return (predictions_list, pcd_list, id_list)

In [ ]:
if inference:
    predictions_list, cloud_list, id_list = model_inference(
        predictor.eval(), testDataLoader, device
    )

In [ ]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 0
    tot = 0
    count = 0

    for j, data in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = (
            data["pointcloud"].to(device).float(),
            data["properties"].to(device),
            data["id"].to(device),
        )
        points, target, ids = (
            data["pointcloud"].to(device).float(),
            data["properties"].to(device),
            data["id"].to(device),
        )
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device("cpu"))
        inputs = points.to(torch.device("cpu"))
        labels = target.to(torch.device("cpu"))
        ids = ids.to(torch.device("cpu"))
        # print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = (
                (labels[i][parameter_id] - outputs[i][parameter_id])
                / labels[i][parameter_id]
            ).item()
            # print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            # print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())

        count += outputs.size(0)
    print(tot / count)

    return predictions_list, inputs_list, label_list, output_list, id_list, cloud_list

In [ ]:
if not inference:
    with torch.no_grad():
        (
            predictions_list,
            inputs_list,
            label_list,
            output_list,
            id_list,
            cloud_list,
        ) = test(predictor.eval(), testDataLoader, device, test_criterion)

    print(len(predictions_list), len(inputs_list))

In [ ]:
if not inference:
    label_list, output_list, id_list = (
        np.array(label_list),
        np.array(output_list),
        np.array(id_list),
    )

In [ ]:
print(id_list[:10])

In [ ]:
# fix negative radii, result of training on noisy data. TODO: figure out why?
for i in range(len(predictions_list)):
    predictions_list[i][0] = abs(predictions_list[i][0])

In [ ]:
print(predictions_list[0])

#### Visually analyse predictions and Fine tune with ICP, calculate chamfer distances

In [ ]:
# evaluate chamfer distance for set of results
def chamfer_evaluate(predictions_list, cloud_list, cat):
    cuda = torch.device("cuda")
    preds_t = torch.tensor(
        predictions_list, requires_grad=True, device=cuda, dtype=torch.float
    )
    cloud_t = torch.tensor(cloud_list, device=cuda, dtype=torch.float)

    chamfer_dists = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False)
    chamfer_dists = chamfer_dists.detach().cpu().numpy()

    return chamfer_dists


# # scaling up and down is required for icp calculations
# def chamfer_evaluate_with_icp(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
#                      cloudCompare_executable, temp_dir, target_dir, sample_size,
#                      threshold, icp_correction = False):

#     preds_list, pcd_list = [], []
#     error_count = 0

#     # get predictions and pcds
#     for i in tqdm(range(len(predictions_list))):
#     #for i in tqdm(range(50)):
#         pcd_id = id_list[i]
#         pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
#         #print(preds, inputs_list[i])

#         preds = scale_preds(preds.tolist(), cat)
#         #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)

#         try:
#             if  icp_correction:
#                 # note: preds are updated in place during ICP
#                 _, _ = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir,
#                                      ifcConvert_executable, cloudCompare_executable, sample_size, threshold, False)

#             preds_list.append(preds)
#             pcd_list.append(pcd)

#         except Exception as e:
#             print("ICP error", pcd_id, e)
#             error_count += 1

#     # calculate chamfer distances
#     cuda = torch.device('cuda')
#     rescaled_preds = [scale_preds(preds, cat, up=0) for preds in preds_list]
#     preds_t = torch.tensor(rescaled_preds, requires_grad=True, device=cuda)
#     cloud_t = torch.tensor(cloud_list, device=cuda)

#     chamfer_dists = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False)
#     chamfer_dists = chamfer_dists.detach().cpu().numpy()

#     print("error_count", error_count)
#     return chamfer_dists

In [ ]:
# dists = chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
#                      cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
dists = chamfer_evaluate(predictions_list, cloud_list, cat)
# if inference:
#     with open(model_path + 'preds_' + cat + '.pkl', 'wb') as f:
#         pickle.dump([predictions_list, id_list, dists], f)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:
# scaling up and down is required for icp calculations
def _visualise_predictions(
    predictions_list,
    cloud_list,
    id_list,
    cat,
    blueprint,
    ifcConvert_executable,
    cloudCompare_executable,
    temp_dir,
    target_dir,
    sample_size,
    threshold,
    icp_correction=False,
):
    preds_list, pcd_list = [], []
    viewer_list, ifc_list = [], []
    error_count = 0

    # get predictions and pcds
    # for i in tqdm(range(len(predictions_list))):
    for i in tqdm(range(50)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0).tolist(), copy.deepcopy(
            predictions_list[i]
        )
        # print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        # print(preds)
        # pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, ext)

        #         try:
        if icp_correction:
            # note: preds are updated in place during ICP
            viewer, ifc = icp_finetuning(
                o3d.utility.Vector3dVector(pcd),
                pcd_id,
                cat,
                preds,
                blueprint,
                temp_dir,
                target_dir,
                ifcConvert_executable,
                cloudCompare_executable,
                sample_size,
                threshold,
                True,
            )
        else:
            # print(type(preds[0]))
            # print("lp", preds)
            viewer, ifc = visualize_predictions(
                [pcd], cat, [preds], blueprint, visualize=True
            )

        preds_list.append(preds)
        pcd_list.append(pcd)
        viewer_list.append(viewer)
        ifc_list.append(ifc)

    #         except Exception as e:
    #             print("ICP error", pcd_id, e)
    #             error_count += 1

    print("error_count", error_count)
    return viewer_list

In [ ]:
# viewers = _visualise_predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
#                      cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
# for v in viewers:
#     print(v)

In [ ]:
# plot_parameter_errors(inputs_list, predictions_list, cat)

#### BP data Visualisation

In [ ]:
# batch_visualise(model_path, blueprint, path, ext, device, ifc=False)

In [ ]:
# merge_clouds(path, 'pipe')

### fine tuning

In [ ]:
print(len(predictions_list))
vis = 1

In [ ]:
# load gmm parameters
gmm_path = Path("gaussians_full.pkl")
with open(gmm_path, "rb") as f:
    means, covs, gmm_ids = pickle.load(f)
    # means, covs = torch.tensor(means).cuda(), torch.tensor(covs).cuda()

# sort gmms by predictions id
sorted_means = []
sorted_covs = []

for i in range(len(id_list)):
    idx = gmm_ids.index(id_list[i])
    sorted_means.append(means[idx])
    sorted_covs.append(covs[idx])

means, covs = torch.tensor(np.array(sorted_means)).cuda(), torch.tensor(np.array(sorted_covs)).cuda()
print(means.shape, covs.shape)

In [ ]:
%autoreload 2
# optimise using mahalanobis distance
print(cat)

if cat == "elbow":
    elbow_fix = False
else:
    elbow_fix = True

limit = 64
cl_v, v, mahal_modified_preds = mahalanobis_fine_tune(
    100,
    0.01,
    predictions_list[:limit],
    cloud_list[:limit],
    means[:limit],
    covs[:limit],
    cat,
    blueprint,
    alpha=3,
    visualise=True,
    elbow_fix=elbow_fix,
    robust=None,
    delta=0.0001,
    chamfer=1000000
)

if cat == "elbow":
    mahal_modified_preds = mahal_modified_preds[0]

In [ ]:
%autoreload 2

# non-robust
# torch.autograd.set_detect_anomaly(False)
print(cat)
if cat == "bend":
    cat = "elbow"

if cat == "elbow":
    elbow_fix = False
else:
    elbow_fix = True

if vis:
    limit = 64
    cl_v_c, v_c, modified_preds = chamfer_fine_tune(
        100,
        0.01,
        predictions_list[:limit],
        cloud_list[:limit],
        cat,
        blueprint,
        alpha=3,
        visualise=True,
        elbow_fix=elbow_fix,
        robust=None,
        delta=0.0001,
    )
else:
    limit = len(predictions_list)
    modified_preds = chamfer_fine_tune(
        100,
        0.01,
        predictions_list[:limit],
        cloud_list[:limit],
        cat,
        blueprint,
        alpha=3,
        visualise=False,
        elbow_fix=elbow_fix,
        robust=None,
        delta=0.0001,
    )
    
if cat == "elbow":
    modified_preds = modified_preds[0]

In [ ]:
# # robust
# #torch.autograd.set_detect_anomaly(False)
# print(cat)
# if cat == 'bend':
#     cat = 'elbow'

# if cat == "elbow":
#     elbow_fix = False
# else:
#     elbow_fix = True

# if vis:
#     limit = 50
#     cl_v, v, modified_preds = chamfer_fine_tune(100, 0.01, predictions_list[:limit],
#                                                 cloud_list[:limit], cat, blueprint,
#                                                 alpha=1, visualise=True, elbow_fix=elbow_fix,
#                                                robust="winsor", delta=0.05,
#                                                 bidirectional_robust=False)
# else:
#     limit = len(predictions_list)
#     modified_preds = chamfer_fine_tune(100, 0.01, predictions_list[:limit],
#                                        cloud_list[:limit], cat, blueprint, alpha=1,
#                                        visualise=False, elbow_fix=elbow_fix,
#                                        robust="winsor", delta=0.02,
#                                       bidirectional_robust=False)

In [ ]:
if vis:
    for i in range(0,10,2):
        print(v[i], v[i+1], v_c[i+1])

In [ ]:
if vis:
    print(v)

In [ ]:
# dists = chamfer_evaluate(modified_preds, cloud_list[:limit], id_list, cat, blueprint,  ifcConvert_executable,
#                      cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
dists = chamfer_evaluate(modified_preds, cloud_list[:limit], cat)

if inference:
    with open(path / ("preds_finetuned_" + "pipe" + ".pkl"), "wb") as f:
        pickle.dump([modified_preds, id_list, dists], f)

plot_error_graph(dists, "Fitting Error", max_val=100)

In [ ]:
# robust
plot_parameter_errors(inputs_list, modified_preds, cat)

In [ ]:
# non robust
plot_parameter_errors(inputs_list, modified_preds, cat)

In [ ]:
for i in range(0, len(v), 2):
    print(v[i], v[i + 1], dists[i])

In [ ]:
node_dict = get_features_from_params(path)

In [ ]:
preds = torch.tensor([predictions_list[1]]).cuda()
print(predictions_list[1])
pcd = generate_tee_cloud_tensor(preds)
# tee = generate_tee_cloud(predictions_list[0])
tee = pcd[0].cpu().numpy()
tee = o3d.utility.Vector3dVector(tee)
tee_cloud = o3d.geometry.PointCloud()
tee_cloud.points = tee
o3d.io.write_point_cloud("tee_cl.pcd", tee_cloud)

In [ ]:
original = cloud_list[2]
points = o3d.utility.Vector3dVector(original.transpose(1, 0))
tee_cloud.points = points
o3d.io.write_point_cloud("tee_cl_inp.pcd", tee_cloud)

In [ ]:
# undo normalisation opf the bp tee dataset created for inference, only for comparison with the inferred tee results
tee_path = "tee_fix/tee/test/"
metadata_file = open("tee_fix/tee/metadata.json", "r")
metadata = json.load(metadata_file)
output_path = "tee_fix/tee/unnormalised/"

files = os.listdir(tee_path)
new_points = []
for f in tqdm(files):
    cloud_data = metadata[f.split(".")[0]]
    points = np.array(o3d.io.read_point_cloud(tee_path + f).points)
    print("a", points[0])
    print(cloud_data["norm_factor"], cloud_data["mean"])
    points *= cloud_data["norm_factor"]
    print("b", points[0])

    for i, pnt in enumerate(points):
        pnt += cloud_data["mean"]
    print("c", points[10])
    new_points.append(points)

new_points = o3d.utility.Vector3dVector(np.concatenate(new_points))
new_cloud = o3d.geometry.PointCloud()
new_cloud.points = new_points
o3d.io.write_point_cloud(output_path + "tee_bp_unnormalised.pcd", new_cloud)

## RANSAC benchmarking

In [ ]:
noise = False
noise_size = 256
benchmark_file = "ransac/benchmark.pkl"
data_path = "ransac/build/output/"
save_path = "ransac/build/input/"
no_lines = 3 if cat == "tee" else 2
params_file = "ransac/build/" + cat + "_params.txt"
cloud = o3d.geometry.PointCloud()

#### Create dataset

In [ ]:
# use dataloader to generate a batch of examples with noise
with torch.no_grad():
    predictor = predictor.eval()
    cloud_list = []
    predictions_list = []
    labels_list = []

    for j, data in tqdm(enumerate(testDataLoader), total=1):
        if j == 10:
            break

        #         # introduce noise
        #         if noise:
        #             noisy_clouds = []
        #             dataset = data['pointcloud'].numpy()
        #             for cl in dataset:
        #                 noisy_clouds.append(add_noise(cl, noise_size, rng))
        #             points = torch.Tensor(np.array(noisy_clouds)).to(device).float()
        #         else:
        points = data["pointcloud"].to(device).float()

        # get predictions
        labels = data["properties"].to(torch.device("cpu"))
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device("cpu"))
        points = points.to(torch.device("cpu"))

        for i in range(outputs.size(0)):
            predictions_list.append(outputs[i].numpy())
            labels_list.append(labels[i].numpy())
            cloud_list.append(points[i].numpy())

predictions_list = np.array(predictions_list)
cloud_list = np.array(cloud_list).transpose((0, 2, 1))
label_list = np.array(labels_list)

# save data
for i, cl in enumerate(cloud_list):
    pnts = o3d.utility.Vector3dVector(cl)
    cloud.points = pnts
    o3d.io.write_point_cloud(save_path + str(i) + ".pcd", cloud)
with open(benchmark_file, "wb") as f:
    pickle.dump([predictions_list, labels_list, cloud_list], f)

In [ ]:
with open(benchmark_file, "rb") as f:
    predictions_list, labels_list, cloud_list = pickle.load(f)

In [ ]:
print(len(predictions_list), predictions_list[0].shape, labels_list[0].shape)

#### check RANSAC results

In [ ]:
# load  parameters
files = os.listdir(data_path)
# print(files)

ransac_f = open(params_file, "r")
ransac_params = ransac_f.readlines()
ransac_params = [x.strip() for x in ransac_params]
# print(ransac_params)

In [ ]:
# find params of cylinder, given set of points and ransac outputs (point on axis, axis, r)
def cylinder_parameters_from_ransac(points, params):
    axis = np.array([params[3], params[4], params[5]])
    axis_a = np.array([params[0], params[1], params[2]])
    r = params[6]

    # find projection of each point on cylinder axis
    dists = []
    for i, p in enumerate(points):
        t = np.dot(axis, axis_a - p) / np.dot(axis, axis)
        dists.append(t)

    # find center, length of cylinder
    min_dist, max_dist = min(dists), max(dists)
    center = (2 * axis_a - (min_dist + max_dist) * axis) / 2
    l = max_dist - min_dist

    #     # debugging
    #     min_p = np.array([min(points[:,0]), min(points[:,1]), min(points[:,2])])
    #     max_p = np.array([max(points[:,0]), max(points[:,1]), max(points[:,2])])
    #     center_p = (min_p + max_p)/2
    #     print("c", center, "c2", center_p, "min", min_p, "max", max_p, axis_a)

    return (axis, r, center, l)


# iterate through testset results, compute error
sorted_labels_list, sorted_preds_list, sorted_ransac_list, sorted_points_list = (
    [],
    [],
    [],
    [],
)

for i in range(0, len(ransac_params), no_lines):
    f_name = ransac_params[i]
    idx = int(f_name.split(".")[0])
    points = np.array(o3d.io.read_point_cloud(save_path + f_name).points)

    if ransac_params[i + 1] != "":
        # load inliers
        primary_inliers = np.array(
            o3d.io.read_point_cloud(data_path + "primary_" + f_name).points
        )

        # convert to model prediction format (axis, radius and position)
        primary_params = [float(j) for j in ransac_params[i + 1].split(",")[:-1]]
        p_axis, p_r, p_center, p_l = cylinder_parameters_from_ransac(
            primary_inliers, primary_params
        )
        # print(len(primary_inliers))
    else:
        # get parameters from MOBB instead
        p_axis, p_l, p_lengths, p_center = get_oriented_bbox_from_points(points)
        p_l *= 2
        p_lengths = np.sort(p_lengths)
        p_r = (p_lengths[0] + p_lengths[1]) / 2
        # p_r, p_l, p_center, p_axis = 0.5, 1, [0., 0.,0.], [1.0, 0.,0.]
        # print("p", p_axis)

    param_array = [p_r, p_l, p_center[0], p_center[1], p_center[2]]
    for i in range(3):
        param_array.append(math.sin(p_axis[i]))
        param_array.append(math.cos(p_axis[i]))
    param_array = np.array(param_array)

    if cat == "tee":
        # load inliers
        if ransac_params[i + 1] != "":
            secondary_inliers = np.array(
                o3d.io.read_point_cloud(data_path + "secondary_" + f_name).points
            )
            # print(len(secondary_inliers))
        else:
            # TODO: take axis perpendicular to primary axis as secondary axis
            pass

        # convert to model prediction format (axis, radius and position)
        secondary_params = [float(j) for j in ransac_params[i + 2].split(",")[:-1]]
        s_axis, s_r, s_center, s_l = cylinder_parameters_from_ransac(
            secondary_inliers, secondary_params
        )

    sorted_ransac_list.append(param_array)
    sorted_preds_list.append(predictions_list[idx])
    sorted_labels_list.append(labels_list[idx])
    sorted_points_list.append(points)
    # print("id", idx, labels_list[idx])
    # print(param_array)

In [ ]:
plot_parameter_errors(sorted_labels_list, sorted_preds_list, cat)

In [ ]:
plot_parameter_errors(sorted_labels_list, sorted_ransac_list, cat)

In [ ]:
# visualize predictions
vis = []
error_count = 0
for i, pr in enumerate(tqdm(sorted_ransac_list[:100])):
    try:
        pr = scale_preds(pr.tolist(), cat)
        v, _ = visualize_predictions([sorted_points_list[i]], cat, [pr], blueprint)
        vis.append(v)
    except:
        error_count += 1

print(error_count)

In [ ]:
vis

In [ ]:
inverted_points_list = [points.transpose((1, 0)) for points in sorted_points_list]

dists = chamfer_evaluate(sorted_preds_list, inverted_points_list, cat)
plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:
dists = chamfer_evaluate(sorted_ransac_list, inverted_points_list, cat)
plot_error_graph(dists, "Binned chamfer loss", max_val=300)